In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from datasets.carla import *
from tools.geometry import *
from tools.utils import *

%matplotlib inline
import matplotlib.pyplot as plt

from numpy.linalg import inv
import random

from tqdm.notebook import tqdm      

In [4]:
def find_center(array):
    indices = np.argwhere(array == 1)

    min_x = np.min(indices[:, 1])
    max_x = np.max(indices[:, 1])
    min_y = np.min(indices[:, 0])
    max_y = np.max(indices[:, 0])

    center_x = (max_x + min_x) / 2
    center_y = (max_y + min_y) / 2

    return center_x, center_y


def find_total_height(array):
    indices = np.where(array == 1)

    min_y = np.min(indices[0])
    max_y = np.max(indices[0])

    total_height = max_y - min_y + 1
    return total_height


def resize_image_by_height(image, new_height):
    width, height = image.size
    aspect_ratio = width / height
    new_width = int(aspect_ratio * new_height)

    resized_image = image.resize((new_width, int(new_height)))
    return resized_image

In [5]:
carla_data = CarlaDataset("../../data/carla/train", False)

In [6]:
animals = [
    "bear",
    "elephant",
    "horse",
    "deer"
]
    
cameras = [
    'left_front_camera',
    'front_camera',
    'right_front_camera',
    'left_back_camera',  
    'back_camera',
    'right_back_camera'
]

sizes = [
    [2.1, 5.5, 3.5],
    [4.3, 10, 6],
    [1.75, 3.4, 3.2],
    [2.4, 4, 3],
]

In [ ]:
data_dir = '../../data/carla/train'
save_dir = '../../data/carla/train_aug_2'

In [ ]:
for i, (images, intrinsics, extrinsics, labels, oods) in enumerate(tqdm(carla_data)):
    agent_number = math.floor(i / carla_data.ticks)
    agent_path = os.path.join(data_dir, f"agents/{agent_number}/")
    save_path = os.path.join(save_dir, f"agents/{agent_number}/")
    index = (i + carla_data.offset) % carla_data.ticks

    os.makedirs(os.path.join(save_path, 'bev_semantic'), exist_ok=True)
    
    a = random.randrange(0, len(animals))
   
    for _ in range(10):
        size = sizes[a]
        trans = [random.randint(5, 40), random.randint(-20, 20), size[2] / 2]
        rot = euler_to_quaternion(0, 0, random.randint(0, 360))
        
        I = intrinsics[0]
        E = np.linalg.inv(extrinsics[0])
        
        bev_ood, cam_ood = draw_box(
            trans, rot, size, I, E,
            dataset='carla',
        )
        
        road = labels[1].numpy()
        road = road[bev_ood.astype(np.bool_)]

        if np.sum(road) > 0:
            break
        
    file = random.choice(os.listdir(f"./pseudo_new/{animals[a]}/"))
    ood = Image.open(f"./pseudo_new/{animals[a]}/{file}")
    
    if random.choice([True, False]):
        trans[0] *= -1              

    for i in range(6):
        sensor_name = cameras[i]
        image = Image.open(os.path.join(agent_path + sensor_name, f'{index}.png'))
        
        I = intrinsics[i]
        E = np.linalg.inv(extrinsics[i])
    
        bev_ood, cam_ood = draw_box(
            trans, rot, size, I, E, 
            dataset='carla',           
        )             
        
        if np.sum(cam_ood) > 0:           
            mean = find_center(cam_ood)
            ood = resize_image_by_height(ood, find_total_height(cam_ood) * 1.2)
            
            x = round(mean[0]) - ood.width // 2
            y = round(mean[1]) - ood.height // 2
        
            image.paste(ood, (x, y), ood)
        
        os.makedirs(os.path.join(save_path, sensor_name), exist_ok=True)
        image.save(os.path.join(save_path + sensor_name, f'{index}.png'))

    label = np.array(Image.open(os.path.join(agent_path + "bev_semantic", f'{index}.png')))
    label[bev_ood == 1, :] = 0
    
    cv2.imwrite(os.path.join(save_path + "bev_semantic", f'{index}.png'), label)
                                         